In [49]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [50]:
"""
DEFINIR VARIABLE PARA CONSEGUIR TODAS LAS URLS DE LA PÁGINA
"""
URLS = [] # Lista vacia que contenga todas las URLS de la página

URL_BASE = "https://www.fatsecret.com.ar/calor%C3%ADas-nutrici%C3%B3n/search?q=9+de+Oro&pg=" # Definimos la URL_BASE, la cual es la misma para todas las paginas

# Hacemos un loop que agregue a la URL_BASE el numero de página.

for num in range(0,271):

    URL = URL_BASE + str(num) 

    URLS.append(URL) # Agregamos a la lista vacia las URLS de todas las página

In [51]:
print(URLS[2])

https://www.fatsecret.com.ar/calor%C3%ADas-nutrici%C3%B3n/search?q=9+de+Oro&pg=2


In [52]:
"""
CREAMOS EL DATAFRAME QUE CONTENERA LOS PRODUCTOS
"""
Tabla_Nutricional_Productos = pd.DataFrame()

In [53]:
"""
LOOP PRINCIPAL PARA CONSEGUIR LA INFORMACIÓN NUTRICIONAL
"""

for url in URLS:
    
    page = requests.get(url)

    pagesoup = BeautifulSoup(page.content, "html.parser")

    tabla = pagesoup.find("td",class_="leftCell")

    Productos = tabla.find_all("a",class_="prominent")
    Producto = [caracter.text for caracter in Productos]
    
    Marcas = tabla.find_all("a",class_="brand")
    Marca = [caracter.text.replace("(","").replace(")","") if len(caracter.text) > 0 else None for caracter in Marcas]
            
    Cantidades = tabla.find_all("div",class_="smallText greyText greyLink")
    Cantidad = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("-", 1)[0] for caracter in Cantidades]
            
    Calorias = tabla.find_all("div", class_="smallText greyText greyLink")
    Caloria = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("-", 1)[-1].split("|")[0].strip().replace("Calorías:", "").strip() 
           for caracter in Calorias]
            
    Grasas = tabla.find_all("div", class_="smallText greyText greyLink")
    Grasa = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 1)[-1].split("|")[0].strip().replace("Grasa:", "").strip() 
         for caracter in Grasas]
            
    Carbohidratos = tabla.find_all("div", class_="smallText greyText greyLink")
    Carbohidrato = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 2)[-1].split("|")[0].strip().replace("Carbh:", "").strip()
                for caracter in Carbohidratos]
            
    Proteinas = tabla.find_all("div", class_="smallText greyText greyLink")
    Proteina = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 3)[-1].split("|")[0].strip().replace("Prot:", "").split("g", 1)[0] + "g".strip() 
            for caracter in Proteinas]
        
    if not Tabla_Nutricional_Productos.empty:
        columnas = ["Producto","Marca","Cantidad(g)","Caloria(kcal)","Grasa(g)","Carbohidrato(g)","Proteina(g)"]
        data_ = list(zip(Producto, Marca, Cantidad, Caloria, Grasa, Carbohidrato, Proteina))
        df = pd.DataFrame(columns=columnas,data=data_)
        Tabla_Nutricional_Productos = pd.concat([Tabla_Nutricional_Productos,df],ignore_index=True)
    else:
        columnas = ["Producto","Marca","Cantidad(g)","Caloria(kcal)","Grasa(g)","Carbohidrato(g)","Proteina(g)"]
        data_ = list(zip(Producto, Marca, Cantidad, Caloria, Grasa, Carbohidrato, Proteina))
        Tabla_Nutricional_Productos = pd.DataFrame(columns=columnas,data=data_)

In [55]:
Tabla_Nutricional_Productos

,Producto,Marca,Cantidad(g),Caloria(kcal),Grasa(g),Carbohidrato(g),Proteina(g)
0,Bizcochos Clasicos,9 de Oro,por 6 unidades (30g),144kcal,"6,70g","18,00g","2,80g"
1,Bizcochos Agridulces,9 de Oro,por 11 unidades (30g),145kcal,"5,70g","21,00g","2,50g"
2,Bizcochos Azucarados,9 de Oro,por 6 unidades (34g),160kcal,"6,40g","23,00g","2,50g"
3,Pepas,9 de Oro,por 2 pepas (30g),97kcal,"3,00g","15,00g","1,00g"
4,Budín de Chocolate,9 de Oro,por 1 rebanada (60g),241kcal,"13,00g","27,00g","5,00g"
...,...,...,...,...,...,...,...
2705,Tableta de Dulce de Leche Suave,Vauquita,por 1 porción (22g),69kcal,"0,00g","16,10g","1,30g"
2706,Fiambre Cocido de Paleta de Cerdo,Vea,por 2 fetas (40g),38kcal,"1,20g","2,00g","4,80g"
2707,Mantequilla de Maní Dulce de Leche,Le-Fit,por 1 cucharada sopera (15g),82kcal,"6,70g","1,60g","3,70g"
2708,Barrita de Arroz Yoghurt de Frutilla,Crowie,por 1 unidad (12g),63kcal,"3,90g","5,90g","1,00g"


In [59]:
Tabla_Nutricional_Productos["Proteina(g)"] = Tabla_Nutricional_Productos["Proteina(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Proteina(g)"] = Tabla_Nutricional_Productos["Proteina(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Grasa(g)"] = Tabla_Nutricional_Productos["Grasa(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Grasa(g)"] = Tabla_Nutricional_Productos["Grasa(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Carbohidrato(g)"] = Tabla_Nutricional_Productos["Carbohidrato(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Carbohidrato(g)"] = Tabla_Nutricional_Productos["Carbohidrato(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Caloria(kcal)"] = Tabla_Nutricional_Productos["Caloria(kcal)"].str.replace("kcal","")
Tabla_Nutricional_Productos["Cantidad(g)"] = Tabla_Nutricional_Productos["Cantidad(g)"].str.extract(r'\((.*?)\)')
Tabla_Nutricional_Productos["Cantidad(g)"] = Tabla_Nutricional_Productos["Cantidad(g)"].str.replace("g","")



In [60]:
Tabla_Nutricional_Productos

,Producto,Marca,Cantidad(g),Caloria(kcal),Grasa(g),Carbohidrato(g),Proteina(g)
0,Bizcochos Clasicos,9 de Oro,30,144,6.70,18.00,2.80
1,Bizcochos Agridulces,9 de Oro,30,145,5.70,21.00,2.50
2,Bizcochos Azucarados,9 de Oro,34,160,6.40,23.00,2.50
3,Pepas,9 de Oro,30,97,3.00,15.00,1.00
4,Budín de Chocolate,9 de Oro,60,241,13.00,27.00,5.00
...,...,...,...,...,...,...,...
2705,Tableta de Dulce de Leche Suave,Vauquita,22,69,0.00,16.10,1.30
2706,Fiambre Cocido de Paleta de Cerdo,Vea,40,38,1.20,2.00,4.80
2707,Mantequilla de Maní Dulce de Leche,Le-Fit,15,82,6.70,1.60,3.70
2708,Barrita de Arroz Yoghurt de Frutilla,Crowie,12,63,3.90,5.90,1.00
